In [1]:
import os
import numpy as np
import time
import socket
from tqdm.auto import tqdm

In [2]:
class C4DController():
    def __init__(self, PORT = 3014):
        # connect to Maya server
        HOST = '127.0.0.1'  # Symbolic name meaning the local host
        #PORT = PORT  # Arbitrary non-privileged port

        ADDR = (HOST, PORT)

        self.client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client.connect(ADDR)

    def SendCommand(self, command: str):
        self.client.sendall(bytes(command, 'utf-8'))

    def RegisterEvent(self):
        self.client.sendall(bytes("c4d.EventAdd()", 'utf-8'))

    def SetOneKeyFrame(self, curve_name: str, frame: int, value: float):
        # Retrieves the current time
        self.SendCommand("keyTime = c4d.BaseTime({}, doc.GetFps())".format(str(frame)))
        self.SendCommand("added = {}.AddKey(keyTime)".format(curve_name))
        self.SendCommand("added[\"key\"].SetValue({}, 0)".format(curve_name))
        self.SendCommand("added[\"key\"].SetInterpolation({},c4d.CINTERPOLATION_SPLINE)".format(curve_name))
        self.SendCommand("{}.SetKeyDefault(doc, added_X[\"nidx\"])".format(curve_name))
        
        # Sets the key to default status AutoTangent etc...

In [9]:
cc = C4DController()

In [ ]:
#cc.SendCommand("global doc")

In [ ]:
cc.SendCommand("print(keyTime.Get())")

In [ ]:
cc.SendCommand("root_name = \"f_avg_root\"")

In [ ]:
cc.SendCommand("doc2 = c4d.documents.GetActiveDocument()")

In [ ]:
cc.SendCommand("c4d.documents.SetActiveDocument(doc2)")

In [ ]:
cc.SendCommand("root = doc2.SearchObject(root_name)")

In [ ]:
cc.SendCommand("keyTime = c4d.BaseTime({}, doc.GetFps())".format("10"))

In [ ]:
cc.SendCommand("root_trX = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_POSITION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_X, c4d.DTYPE_REAL, 0)))")

In [ ]:
cc.SendCommand("root_trY = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_POSITION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Y, c4d.DTYPE_REAL, 0)))")

In [ ]:
cc.SendCommand("root_trZ = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_POSITION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Z, c4d.DTYPE_REAL, 0)))")

In [ ]:
cc.SendCommand("e = 1{}f= 2".format("\n"))

In [ ]:
ll = ["e = 5","f=412"]

In [ ]:
cc.SendCommand("\n".join(preparation_command_0))

In [3]:
preparation_command_0 = ["import c4d",
"# set documentation",
"doc = c4d.documents.GetActiveDocument()",
"root_name = \"f_avg_root\"",
"root = doc.SearchObject(root_name)",
"root.SetRotationOrder(5)",
"# Creates the track in memory. Defined by it's DESCID    ",
"root_trX = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_POSITION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_X, c4d.DTYPE_REAL, 0)))",
"root_trY = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_POSITION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Y, c4d.DTYPE_REAL, 0)))",
"root_trZ = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_POSITION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Z, c4d.DTYPE_REAL, 0)))",
"# Creates the track in memory. Defined by it's DESCID    ",
"root_rX = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_REL_ROTATION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_X, c4d.DTYPE_REAL, 0)))",
"root_rY = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_REL_ROTATION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Y, c4d.DTYPE_REAL, 0)))",
"root_rZ = c4d.CTrack(root, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_REL_ROTATION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Z, c4d.DTYPE_REAL, 0)))",
"# Gets Curves for the track",
"root_curveX = root_trX.GetCurve()",
"root_curveY = root_trY.GetCurve()",
"root_curveZ = root_trZ.GetCurve()",
"# Gets Curves for the track",
"root_curveRX = root_rX.GetCurve()",
"root_curveRY = root_rY.GetCurve()",
"root_curveRZ = root_rZ.GetCurve()"]

In [4]:
# Get skeleton information from https://meshcapade.wiki/SMPL#smpl-x
SMPL_SKELETON = {
    0: 'Pelvis',     3: 'Spine1',       6: 'Spine2',    9: 'Spine3',    12: 'Neck',     15: 'Head',
     1: 'L_Hip',      4: 'L_Knee',       7: 'L_Ankle',  10: 'L_Foot',
     2: 'R_Hip',      5: 'R_Knee',       8: 'R_Ankle',  11: 'R_Foot',
    13: 'L_Collar',  16: 'L_Shoulder',  18: 'L_Elbow',  20: 'L_Wrist',
    14: 'R_Collar',  17: 'R_Shoulder',  19: 'R_Elbow',  21: 'R_Wrist',
    22: 'L_Hand',
    23: 'R_Hand'
}

SMPL_H_SKELETON = {
    0: 'Pelvis',     3: 'Spine1',       6: 'Spine2',    9: 'Spine3',    12: 'Neck',     15: 'Head',
    1: 'L_Hip',      4: 'L_Knee',       7: 'L_Ankle',  10: 'L_Foot',
    2: 'R_Hip',      5: 'R_Knee',       8: 'R_Ankle',  11: 'R_Foot',
    13: 'L_Collar',  16: 'L_Shoulder',  18: 'L_Elbow',  20: 'L_Wrist',
    14: 'R_Collar',  17: 'R_Shoulder',  19: 'R_Elbow',  21: 'R_Wrist',
    22: 'lindex0',  23: 'lindex1',  24: 'lindex2',
    25: 'lmiddle0', 26: 'lmiddle1', 27: 'lmiddle2',
    28: 'lpinky0',  29: 'lpinky1',  30: 'lpinky2',
    31: 'lring0',   32: 'lring1',   33: 'lring2',
    34: 'lthumb0',  35: 'lthumb1',  36: 'lthumb2',
    37: 'rindex0',  38: 'rindex1',  39: 'rindex2',
    40: 'rmiddle0', 41: 'rmiddle1', 42: 'rmiddle2',
    43: 'rpinky0',  44: 'rpinky1',  45: 'rpinky2',
    46: 'rring0',   47: 'rring1',   48: 'rring2',
    49: 'rthumb0',  50: 'rthumb1',  51: 'rthumb2'
}

In [5]:
joint_name = "f_avg_" + SMPL_H_SKELETON[1]

In [6]:
joint_name

'f_avg_L_Hip'

In [7]:
def RegisterJoints(joint):
    register_command = [
        "{} = doc.SearchObject(\"{}\")".format(joint,joint),
        "{}.SetRotationOrder(5)".format(joint),
        "{}_rX = c4d.CTrack({}, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_REL_ROTATION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_X, c4d.DTYPE_REAL, 0)))".format(joint, joint),
        "{}_rY = c4d.CTrack({}, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_REL_ROTATION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Y, c4d.DTYPE_REAL, 0)))".format(joint, joint),
        "{}_rZ = c4d.CTrack({}, c4d.DescID(c4d.DescLevel(c4d.ID_BASEOBJECT_REL_ROTATION, c4d.DTYPE_VECTOR, 0), c4d.DescLevel(c4d.VECTOR_Z, c4d.DTYPE_REAL, 0)))".format(joint, joint),
        "{}_curveRX = {}_rX.GetCurve()".format(joint, joint),
        "{}_curveRY = {}_rY.GetCurve()".format(joint, joint),
        "{}_curveRZ = {}_rZ.GetCurve()".format(joint, joint),
    ]
    
    cc.SendCommand("\n".join(register_command))

In [10]:
for i in tqdm(range(1, len(SMPL_H_SKELETON))):
    joint_name = "f_avg_" + SMPL_H_SKELETON[i]
    RegisterJoints(joint_name)
    time.sleep(0.05)

  0%|          | 0/51 [00:00<?, ?it/s]

In [11]:
def SetOneKeyFrame(curve_name: str, frame: int, value: float):
    # Retrieves the current time
    frame_command = ["keyTime = c4d.BaseTime({}, doc.GetFps())".format(str(frame)),
    "added = {}.AddKey(keyTime)".format(curve_name),
    "added[\"key\"].SetValue({}, {})".format(curve_name, str(value)),
    "added[\"key\"].SetInterpolation({},c4d.CINTERPOLATION_SPLINE)".format(curve_name),
    "{}.SetKeyDefault(doc, added[\"nidx\"])".format(curve_name)]
    
    # print
    cc.SendCommand("\n".join(frame_command))

In [12]:
SetOneKeyFrame("root" + "_curveRX", 30, 0)

In [ ]:
cc.SendCommand("{}.InsertTrackSorted({}_rX)".format(joint_name, joint_name))

In [ ]:
cc.SendCommand("c4d.EventAdd()")

In [13]:
#amass_npz_fname = '../../../thirdParty/amass/support_data/github_data/amass_sample.npz' # the path to body data
amass_npz_fname = '../../../thirdParty/accad/C2 - Run to stand_poses.npz' # the path to body data
bdata = np.load(amass_npz_fname)

In [14]:
bdata['poses'][0]

array([ 1.54106054,  0.59912531,  0.54576866, -0.65198941, -0.19927297,
       -0.07908039,  0.55414857, -0.19172799, -0.00372826,  0.04291578,
        0.32047213, -0.12076745,  0.3368134 , -0.04331569, -0.11839403,
        1.12465086,  0.11197466,  0.04326428,  0.08771633,  0.02837623,
        0.04840781, -0.03945161,  0.14846137,  0.14807201,  0.05118354,
       -0.05656281,  0.12596686, -0.04036598,  0.18163919, -0.002879  ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.01366199, -0.22342897,  0.02492875,  0.02110526,
        0.04735527, -0.16303656, -0.01276061,  0.19015719,  0.21894259,
       -0.25376615, -0.29326656,  0.16493634,  0.33925243,  0.20912539,
       -0.56528983,  0.05857785,  0.06295114,  0.79395729,  0.69919005,
       -1.84615729,  0.40790663,  0.21165528,  1.57535784, -0.54417244,
       -0.14943051, -0.06103526,  0.07610893, -0.12874111,  0.04456872,
        0.07671561,  0.11167872,  0.04289217, -0.41644184,  0.10

In [15]:
joints_info_list = SMPL_H_SKELETON
body_data = bdata

In [19]:
for i in tqdm(range(1, 22)):
    joint_name = "f_avg_" + SMPL_H_SKELETON[i]
    for frame in range(0,11,10):
        # get rotation from body data
        rotateX, rotateY, rotateZ = body_data['poses'][frame][i*3:(i+1)*3]

        # get joint name
        joint_name = "f_avg_" + joints_info_list[i]
        SetOneKeyFrame(joint_name + "_curveRX", frame, rotateX)
        time.sleep(0.05)
        SetOneKeyFrame(joint_name + "_curveRY", frame, rotateY)
        time.sleep(0.05)
        SetOneKeyFrame(joint_name + "_curveRZ", frame, -rotateZ)
        time.sleep(0.05)
        
    cc.SendCommand("{}.InsertTrackSorted({}_rX)".format(joint_name, joint_name))
    time.sleep(0.05)
    cc.SendCommand("{}.InsertTrackSorted({}_rY)".format(joint_name, joint_name))
    time.sleep(0.05)
    cc.SendCommand("{}.InsertTrackSorted({}_rZ)".format(joint_name, joint_name))
    time.sleep(0.05)

  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
cc.SendCommand("c4d.EventAdd()")

In [ ]:
body_data['poses'][frame]

In [17]:
frame = 0

In [18]:
for i in tqdm(range(1, 22)):
    joint_name = "f_avg_" + SMPL_H_SKELETON[i]
    rotateX, rotateY, rotateZ = body_data['poses'][frame][i*3:(i+1)*3]
    
    print(joint_name, rotateX * 57, rotateY * 57, rotateZ * 57)

  0%|          | 0/21 [00:00<?, ?it/s]

f_avg_L_Hip -37.16339617769358 -11.358559318797205 -4.507582380838112
f_avg_R_Hip 31.58646852086978 -10.928495416081912 -0.2125105729969213
f_avg_Spine1 2.446199282571145 18.266911432803713 -6.883744459074074
f_avg_L_Knee 19.198363943963564 -2.4689940653315157 -6.7484595052656475
f_avg_R_Knee 64.1050990565303 6.382555463752623 2.4660640601283674
f_avg_Spine2 4.999830633637596 1.6174451071620526 2.7592449572132565
f_avg_L_Ankle -2.2487419984471906 8.462298041543097 8.440104541027088
f_avg_R_Ankle 2.9174617456602956 -3.2240799280141035 7.1801110069486676
f_avg_Spine3 -2.3008611281771447 10.353433677877582 -0.16410287643809085
f_avg_L_Foot 0.0 0.0 0.0
f_avg_R_Foot 0.0 0.0 0.0
f_avg_Neck -0.7787333457612976 -12.735451522153458 1.4209385721796866
f_avg_L_Collar 1.2030000578736724 2.6992504905037977 -9.293084186570448
f_avg_R_Collar -0.7273545633180499 10.838959818095445 12.479727349138695
f_avg_Head -14.464670352731194 -16.71619384888073 9.40137124537076
f_avg_L_Shoulder 19.33738835586455 1